In [1]:
import pandas as pd
import numpy as np
import json
import ctypes
import hashlib
from datetime import datetime
from bson import json_util

PATH = '/Users/Dario/Google Drive/DS/First Year - Secon Semester/DM/homeworks/hw_1_&_2/data/'

In [2]:
customers = pd.read_csv(PATH + 'olist_customers_dataset.csv')
items = pd.read_csv(PATH + 'olist_order_items_dataset.csv')
payments = pd.read_csv(PATH + 'olist_order_payments_dataset.csv')
reviews = pd.read_csv(PATH + 'olist_order_reviews_dataset.csv')
orders = pd.read_csv(PATH + 'olist_orders_dataset.csv')
products = pd.read_csv(PATH + 'olist_products_dataset.csv')
sellers = pd.read_csv(PATH + 'olist_sellers_dataset.csv')

In [3]:
# Convert dates:

In [4]:
cols = ['order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 
        'order_estimated_delivery_date']

for col in cols:
    orders[col]=orders[col].apply(lambda x: json.dumps(datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), default = json_util.default)[10:-1] if type(x) != float else np.NaN)

items['shipping_limit_date'] = items['shipping_limit_date'].apply(lambda x: json.dumps(datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), default = json_util.default)[10:-1] if type(x) != float else np.NaN)

cols = ['review_creation_date','review_answer_timestamp']
for col in cols:
    reviews[col]=reviews[col].apply(lambda x: json.dumps(datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), default = json_util.default)[10:-1] if type(x) != float else np.NaN)

# Work on Customers Aggregate

In [5]:
# Merge customer and orders:

In [6]:
co = customers.merge(orders, on = 'customer_id')

In [7]:
len(co)

99441

In [8]:
# Generate unique id for address:

In [9]:
def hashFor(data):
    # Prepare the project id hash
    hashId = hashlib.md5()

    hashId.update(repr(data).encode('utf-8'))

    return hashId.hexdigest()

In [10]:
cols = ['customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
co['address_id'] = co[cols].apply(lambda x: hashFor(list(x)), axis = 1)

In [11]:
# get the column address:

In [12]:
cols = ['address_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state']
co['address'] = co[cols].apply(lambda x: {k:v for k,v in list(zip(cols, x))}, axis = 1)

In [13]:
# Group by the custumer unique id and get the orders each customer did and its addresses:

In [14]:
cog = co.groupby('customer_unique_id')['address'].apply(lambda x: list(x))
cog = cog.reset_index()

In [15]:
# Remove duplicated address:

In [16]:
def remdup(lst):
    v = []
    for el in lst:
        if el not in v:
            v.append(el) 
    return v

In [17]:
cog['address'] = cog['address'].apply(lambda x: remdup(x))

In [18]:
# Save customers json file:

In [19]:
'''
customers_d = cog.to_dict(orient='records')
with open('custmers.json', 'w') as fp:
    json.dump(customers_d, fp)
'''

"\ncustomers_d = cog.to_dict(orient='records')\nwith open('custmers.json', 'w') as fp:\n    json.dump(customers_d, fp)\n"

# Work on delivery info:

In [20]:
# Create delivery info:

In [21]:
cols = ['address', 'order_delivered_carrier_date','order_delivered_customer_date', 'order_estimated_delivery_date']
cold = ['shipping_address', 'order_delivered_carrier_date','order_delivered_customer_date', 'order_estimated_delivery_date']
co['order_delivery_info'] = co[cols].apply(lambda x: {k: v for k,v in list(zip(cold, list(x)))}, axis = 1)

In [22]:
# DF order_id, delivery_info:

In [23]:
od = co[['order_id', 'order_delivery_info']]

In [24]:
od.head()

,order_id,order_delivery_info
0,00e7ee1b050b8499577073aeb2a297a1,{'shipping_address': {'address_id': 'c36b63808...
1,29150127e6685892b6eab3eec79f59c7,{'shipping_address': {'address_id': 'b7d131545...
2,b2059ed67ce144a36e2aa97d2c9e9ad2,{'shipping_address': {'address_id': '1b11ad921...
3,951670f92359f4fe4a63112aa7306eba,{'shipping_address': {'address_id': 'ea9a05819...
4,6b7d50bd145f6fc7f33cebabd7e49d0f,{'shipping_address': {'address_id': 'ea65651ce...


In [25]:
od['order_delivery_info'][0]

{'shipping_address': {'address_id': 'c36b63808d3356977d43e03fc716e750',
  'customer_zip_code_prefix': 14409,
  'customer_city': 'franca',
  'customer_state': 'SP'},
 'order_delivered_carrier_date': '1495536477000',
 'order_delivered_customer_date': '1495708535000',
 'order_estimated_delivery_date': '1496620800000'}

# Work on items in orders:

In [26]:
oi = orders.merge(items, on = 'order_id', how = 'left')
oi = oi.merge(products, on = 'product_id', how = 'left')

In [27]:
# Transform product information in a dictionary:

In [28]:
cols = ['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm', 'price', 'freight_value', 
        'shipping_limit_date', 'seller_id']
product_d = oi[cols].apply(lambda x: {k: v for k, v in list(zip(cols, list(x)))}, axis = 1)
oi['order_items'] = product_d  

In [29]:
# Get for each orders the product solds

In [30]:
op = oi.groupby('order_id')['order_items'].apply(lambda x: list(x))
op = op.reset_index()
op = op[['order_id', 'order_items']]

In [31]:
op.head()

,order_id,order_items
0,00010242fe8c5a6d1ba2dd792cb16214,[{'product_id': '4244733e06e7ecb4970a6e2683c13...
1,00018f77f2f0320c557190d7a144bdd3,[{'product_id': 'e5f2d52b802189ee658865ca93d83...
2,000229ec398224ef6ca0657da4fc703e,[{'product_id': 'c777355d18b72b67abbeef9df44fd...
3,00024acbcdf0a6daa1e931b038114c75,[{'product_id': '7634da152a4610f1595efa32f1472...
4,00042b26cf59d7ce69dfabb4e55b4fd9,[{'product_id': 'ac6c3623068f30de03045865e4e10...


# Work on payments in orders

In [32]:
opy = orders.merge(payments, on = 'order_id', how = 'left')

In [33]:
cols = ['payment_type', 'payment_installments', 'payment_value']
payments_d = opy[cols].apply(lambda x: {k: v for k, v in list(zip(cols, list(x)))}, axis = 1)
opy['order_payments'] = payments_d 

In [34]:
opy = opy.groupby('order_id')['order_payments'].apply(lambda x: list(x))
opy = opy.reset_index()

In [35]:
opy.head()

,order_id,order_payments
0,00010242fe8c5a6d1ba2dd792cb16214,"[{'payment_type': 'credit_card', 'payment_inst..."
1,00018f77f2f0320c557190d7a144bdd3,"[{'payment_type': 'credit_card', 'payment_inst..."
2,000229ec398224ef6ca0657da4fc703e,"[{'payment_type': 'credit_card', 'payment_inst..."
3,00024acbcdf0a6daa1e931b038114c75,"[{'payment_type': 'credit_card', 'payment_inst..."
4,00042b26cf59d7ce69dfabb4e55b4fd9,"[{'payment_type': 'credit_card', 'payment_inst..."


# Work on reviews in orders:

In [36]:
orw = orders.merge(reviews, on = 'order_id', how = 'left')

In [37]:
cols = ['review_id', 'review_score', 'review_comment_title','review_comment_message', 'review_creation_date',
       'review_answer_timestamp']
orw['order_reviews'] = orw[cols].apply(lambda x: {k: v for k, v in list(zip(cols, list(x)))}, axis = 1)

In [38]:
orw = orw.groupby('order_id')['order_reviews'].apply(lambda x: list(x))
orw = orw.reset_index()

In [39]:
orw.head()

,order_id,order_reviews
0,00010242fe8c5a6d1ba2dd792cb16214,[{'review_id': '97ca439bc427b48bc1cd7177abe713...
1,00018f77f2f0320c557190d7a144bdd3,[{'review_id': '7b07bacd811c4117b742569b04ce35...
2,000229ec398224ef6ca0657da4fc703e,[{'review_id': '0c5b33dea94867d1ac402749e5438e...
3,00024acbcdf0a6daa1e931b038114c75,[{'review_id': 'f4028d019cb58564807486a6aaf338...
4,00042b26cf59d7ce69dfabb4e55b4fd9,[{'review_id': '940144190dcba6351888cafa43f3a3...


# All info together:

In [40]:
ords = co[['order_id', 'customer_unique_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at']]
ords = ords.merge(od, on = 'order_id', how = 'left')
ords = ords.merge(op, on = 'order_id', how = 'left')
ords = ords.merge(opy, on = 'order_id', how = 'left')
ords = ords.merge(orw, on = 'order_id', how = 'left')

In [41]:
ords.head()

,order_id,customer_unique_id,order_status,order_purchase_timestamp,order_approved_at,order_delivery_info,order_items,order_payments,order_reviews
0,00e7ee1b050b8499577073aeb2a297a1,861eff4711a542e4b93843c6dd7febb0,delivered,1494947135000,1494948132000,{'shipping_address': {'address_id': 'c36b63808...,[{'product_id': 'a9516a079e37a9c9c36b9b78b1016...,"[{'payment_type': 'credit_card', 'payment_inst...",[{'review_id': '88b8b52d46df026a9d1ad2136a59b3...
1,29150127e6685892b6eab3eec79f59c7,290c77bc529b7ac935b93aa66c333dc3,delivered,1515790104000,1515790712000,{'shipping_address': {'address_id': 'b7d131545...,[{'product_id': '4aa6014eceb682077f9dc4bffebc0...,"[{'payment_type': 'credit_card', 'payment_inst...",[{'review_id': '02fc48a9efa3e3d0f1a8ea26507eee...
2,b2059ed67ce144a36e2aa97d2c9e9ad2,060e732b5b29e8181a18229c7b0b2b5e,delivered,1526746065000,1526833150000,{'shipping_address': {'address_id': '1b11ad921...,[{'product_id': 'bd07b66896d6f1494f5b86251848c...,"[{'payment_type': 'credit_card', 'payment_inst...",[{'review_id': '5ad6695d76ee186dc473c42706984d...
3,951670f92359f4fe4a63112aa7306eba,259dac757896d24d7702b9acbbff3f3c,delivered,1520957198000,1520962159000,{'shipping_address': {'address_id': 'ea9a05819...,[{'product_id': 'a5647c44af977b148e0a3a4751a09...,"[{'payment_type': 'credit_card', 'payment_inst...",[{'review_id': '059a801bb31f6aab2266e672cab87b...
4,6b7d50bd145f6fc7f33cebabd7e49d0f,345ecd01c38d18a9036ed96c73b8d066,delivered,1532857890000,1532859009000,{'shipping_address': {'address_id': 'ea65651ce...,[{'product_id': '9391a573abe00141c56e38d84d7d5...,"[{'payment_type': 'credit_card', 'payment_inst...",[{'review_id': '8490879d58d6c5d7773f2739a03f08...


In [42]:
# Save orders json file:

In [43]:
'''
orders_d = ords.to_dict(orient='records')
with open('orders.json', 'w') as fp:
    json.dump(orders_d, fp)
'''

"\norders_d = ords.to_dict(orient='records')\nwith open('orders.json', 'w') as fp:\n    json.dump(orders_d, fp)\n"

# Sellers Data:

In [44]:
# Generate address column:

In [45]:
cols = ['seller_zip_code_prefix', 'seller_city', 'seller_state']
sellers['seller_address'] = sellers[cols].apply(lambda x: {k: v for k, v in list(zip(cols, x))}, axis = 1)

In [46]:
# Save json

In [47]:
'''
seller_d = sellers[['seller_id', 'seller_address']].to_dict(orient = 'record')
with open('sellers.json', 'w') as fp:
    json.dump(seller_d, fp)
'''

"\nseller_d = sellers[['seller_id', 'seller_address']].to_dict(orient = 'record')\nwith open('sellers.json', 'w') as fp:\n    json.dump(seller_d, fp)\n"

In [ ]:
orders_d = ords.iloc[1,:].to_dict(orient='records')
json.dumps(orders_d, fp)